In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/math-qsa-dataset/train.csv
/kaggle/input/math-qsa-dataset/test.csv


## Datasets


In [ ]:
train = pd.read_csv('/kaggle/input/math-qsa-dataset/train.csv')
test = pd.read_csv('/kaggle/input/math-qsa-dataset/test.csv')

In [ ]:
train.head()

,problem,level,type,solution,answer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132


In [ ]:
test.head()

,problem,level,type,solution,answer
0,How many four-digit integers are less than 800...,Level 3,Prealgebra,"We must count the numbers in the list \[2001, ...",5999
1,What is the value of $$\left( \sqrt{100} - \sq...,Level 2,Prealgebra,"Calculating, $(\sqrt{100}-\sqrt{36})^2 = (10-6...",16
2,Each good worker can paint my new house alone ...,Level 5,Prealgebra,Each good worker can paint $1/12$ of my house ...,3
3,"At the national curling championships, there a...",Level 5,Prealgebra,"For each participant, there are 8 opponents to...",162
4,What is the least positive integer value of $x...,Level 2,Prealgebra,We isolate $x$ by subtracting $2x$ from each s...,2


## Models

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import keras
import keras_nlp

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

from transformers import GemmaModel, GemmaConfig, GemmaForCausalLM, FlaxGemmaForCausalLM, AutoTokenizer, AutoModelForCausalLM

In [ ]:
pip install -U transformers datasets accelerate peft trl bitsandbytes wandb

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

from tqdm.notebook import tqdm
tqdm.pandas()

2024-06-15 03:54:53.837952: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 03:54:53.838087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 03:54:53.994817: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# torch_dtype = torch.float16
# attn_implementation = "eager"

In [ ]:
base_model = "nvidia/OpenMath-CodeLlama-7b-Python-hf"

In [ ]:
# # QLoRA config
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch_dtype,
#     bnb_4bit_use_double_quant=True,
# )

# # LoRA config
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
# )

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float32
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nvidia/OpenMath-CodeLlama-7b-Python-hf")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## making template

In [ ]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""

In [ ]:
def is_integer(text):
    try:
        if int(text) >= 0:
            return True
        else:
            return False
    except ValueError:
        return False

In [ ]:
train["is_integer"] = train.answer.map(is_integer)
train = train[train.is_integer].reset_index(drop=True)
train.head(2)

,problem,level,type,solution,answer,is_integer
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3,True
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3,True


In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
train["prompt"] = train.progress_apply(lambda row: template.format(problem=row.problem,
                                                             solution=f"{row.solution}\n\nAnswer:\n{row.answer}"),
                                                             axis=1)
data = train.prompt.tolist()

  0%|          | 0/4456 [00:00<?, ?it/s]

In [ ]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
sample = data[12]

# Give colors to Instruction, Response and Category
sample = colorize_text(sample)

# Show sample in markdown
display(Markdown(sample))



**<font color='blue'>Role:</font>**
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.



**<font color='yellow'>Instruction:</font>**
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.



**<font color='red'>Problem:</font>**
What is the largest positive multiple of $12$ that is less than $350?$



**<font color='cyan'>Solution:</font>**
Dividing $350$ by $12$ gives a quotient $29$ with a remainder of $2$. In other words, \[350=12\cdot29+2.\]Thus, $29\cdot12=\boxed{348}$ is the largest multiple of $12$ which is less than $350.$



**<font color='green'>Answer:</font>**
348

In [ ]:
train.head()

,problem,level,type,solution,answer,is_integer,prompt
0,The United States Postal Service charges an ex...,Level 3,Prealgebra,We calculate the desired ratio for each envelo...,3,True,Role:\nYou are an advanced AI system with exce...
1,How many integers between 1000 and 2000 have a...,Level 4,Prealgebra,"A number with 15, 20 and 25 as factors must be...",3,True,Role:\nYou are an advanced AI system with exce...
2,"Given that $n$ is an integer and $0 < 4n <30$,...",Level 2,Prealgebra,"Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...",28,True,Role:\nYou are an advanced AI system with exce...
3,How many integers between $100$ and $150$ have...,Level 4,Prealgebra,We will break up the problem into cases based ...,18,True,Role:\nYou are an advanced AI system with exce...
4,Regular pentagon $ABCDE$ and regular hexagon $...,Level 4,Prealgebra,We know that the sum of the degree measures of...,132,True,Role:\nYou are an advanced AI system with exce...


In [ ]:
row = train.iloc[12]

# Generate Prompt using template
prompt = template.format(
    problem=tokenizer(row["problem"], return_tensors="pt"),
    solution="",
)

## Trial 2

In [ ]:
template = """Role:\nYou are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.\n\nInstruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.\n\nProblem:\n{problem}\n\nSolution:\n{solution}"""


In [ ]:
problem_data = train.iloc[12]
problem = train['problem']
solution = train.get('solution', '')

In [ ]:
# 템플릿을 이용해 프롬프트 생성
input_text = template.format(problem=problem, solution=solution)

In [ ]:
# 입력을 토큰화
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

# 모델로부터 답 생성
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

# 생성된 답을 디코딩
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 결과 출력
print(f"**생성된 답변:**\n{generated_text}\n")

**생성된 답변:**
Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Problem:
0       The United States Postal Service charges an ex..

In [ ]:
def colorize_text(text):
    for word, color in zip(["Role", "Instruction", "Problem", "Solution", "Answer"],
                           ["blue", "yellow", "red", "cyan", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
# Colorize
output = colorize_text(generated_text)

# Display in markdown
display(Markdown(generated_text))

Role:
You are an advanced AI system with exceptional mathematical reasoning and problem-solving capabilities, specifically designed to solve tricky math problems (whose answer is a non-negative integer) written in LaTeX format from the AI Mathematical Olympiad (AIMO) competition. Your task is to accurately analyze and solve intricate mathematical problems, demonstrating a deep understanding of mathematical concepts and a strong ability to apply logical reasoning strategies.

Instruction:
1. Carefully read and comprehend the problem statement provided in the "Problem" section.
2. In the "Solution" section, provide a solution of the problem with detailed explanation of your logical reasoning process. Keep in mind that answer must be a non-negative integer number.
3. At the end, create a "Answer" section where you will state only the final numerical or algebraic answer, without any additional text or narrative.

Problem:
0       The United States Postal Service charges an ex...
1       How many integers between 1000 and 2000 have a...
2       Given that $n$ is an integer and $0 < 4n <30$,...
3       How many integers between $100$ and $150$ have...
4       Regular pentagon $ABCDE$ and regular hexagon $...
                              ...                        
7493    Let $z$ be a complex number with $|z| = \sqrt{...
7494    Let $a$ and $b$ be real numbers.  Find the max...
7495    In the complex plane, $z,$ $z^2,$ $z^3$ form, ...
7496    There is a smallest positive real number $a$ s...
7497    A function $f$ has domain $[0,2]$ and range $[...
Name: problem, Length: 7498, dtype: object

Solution:
0       We calculate the desired ratio for each envelo...
1       A number with 15, 20 and 25 as factors must be...
2       Dividing by $4$, we have $0<n<7\frac{1}{2}$. T...
3       We will break up the problem into cases based ...
4       We know that the sum of the degree measures of...
                              ...                        
7493    Let $z = x + yi,$ where $x$ and $y$ are real n...
7494    By the Cauchy-Schwarz Inequality,\n\[(a \cos \...
7495    First, consider the case where $z$ is between ...
7496    Let $r,$ $s,$ $t$ be the real roots, so\n\[r^3...
7497    For $g(x) = 1 - f(x + 1)$ to be defined, we ne...
Name: solution, Length: 7498, dtype: object
<ipython-input-1-465354726d9d>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
</llm-code-output>
So we can state that there are $\boxed{6}$ integers between 1000 and 2000 that have a 15, 20, or 25 as a factor.